In [ ]:
'''
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install seqeval
'''

In [ ]:
import json
with open(r"NER_dataset/discharge_res.json","r",encoding="utf-8") as f:
    dataset=json.load(f)


In [ ]:
from datasets import Dataset,DatasetDict
import pandas as pd

train_dataset=pd.DataFrame(dataset)
train_dataset = Dataset.from_dict(dataset)
dataset=DatasetDict({"train":train_dataset,"validation":train_dataset})
dataset

In [ ]:
from transformers import AutoTokenizer
import os
dirpath=os.environ["HF_HOME"]
checkpoint="bert-base-chinese"
tokenizer=AutoTokenizer.from_pretrained(checkpoint,cache_dir=dirpath)

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            #如果不是None则为其对应的word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
    
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)


In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [ ]:
discharge_label=["O","B-G","I-G","B-A","I-A","B-N","I-N","B-C","I-C","B-L","I-L","B-D","I-D","B-IN","I-IN","B-OUT","I-OUT"]
#出院小结标注对应字典
token2label={"无":"O","性别B":"B-G","性别I":"I-G","年龄B":"B-A","年龄I":"I-A","医院名称B":"B-N","医院名称I":"I-N","组织机构代码B":"B-C","组织机构代码I":"I-C","医疗机构类型B":"B-L","医疗机构类型I":"I-L","住院天数B":"B-D","住院天数I":"I-D","入院日期B":"B-IN","入院日期I":"I-IN","出院日期B":"B-OUT","出院日期I":"I-OUT"}
id2label = {str(i): label for i, label in enumerate(discharge_label)}
label2id = {v: k for k, v in id2label.items()}
'''G代表性别、A代表年龄、N代表医院名称、C代表组织机构代码、L代表医疗机构类型、
IN代表入院日期、OUT代表出院日期、D代表住院天数、'''
print(id2label)
print(label2id)

In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
    cache_dir=dirpath,
)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 96
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[discharge_label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [discharge_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from tqdm.auto import tqdm
import torch
from datasets import load_metric

metric = load_metric("seqeval")
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        
        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)
        
        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

In [ ]:
tokenizer.push_to_hub("dis-ner")
model.push_to_hub("dis-ner")